In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [20]:
df = pd.read_csv('Eluvio_DS_Challenge.csv', parse_dates=['date_created'])

In [4]:
#pd.to_datetime(df.date_created.astype(str))
df.sort_values('up_votes', ascending=False)

,time_created,date_created,up_votes,down_votes,title,over_18,author,category
377200,1434818471,2015-06-20,21253,0,A biotech startup has managed to 3-D print fak...,False,KRISHNA53,worldnews
391415,1440421079,2015-08-24,13435,0,Twitter has forced 30 websites that archive po...,False,joeyoungblood,worldnews
450818,1459706506,2016-04-03,13244,0,2.6 terabyte leak of Panamanian shell company ...,False,mister_geaux,worldnews
391318,1440367768,2015-08-23,12333,0,The police officer who leaked the footage of t...,False,navysealassulter,worldnews
390252,1439939168,2015-08-18,11288,0,Paris shooting survivor suing French media for...,False,seapiglet,worldnews
...,...,...,...,...,...,...,...,...
220099,1381644882,2013-10-13,0,0,Chinese Premier Li Keqiang has been granted th...,False,davidreiss666,worldnews
431262,1453107414,2016-01-18,0,0,Taliban denies role in latest Afghanistan bombing,False,raziinuaq,worldnews
220101,1381645228,2013-10-13,0,0,Possible lead in Madeleine McCann case,False,CristabelYYC,worldnews
431259,1453107145,2016-01-18,0,0,Suicide Attack Kills at Least 13 in Afghanistan,False,middleeastnewsman,worldnews


In [5]:
def main(spark):
    df = spark.read.csv('Eluvio_DS_Challenge.csv', schema='time_created INT, date_created DATE, up_votes INT, down_votes INT, title STRING, over_18 BOOLEAN, author STRING, category STRING', header=True)
    print('Printing df inferred schema')
    df.printSchema()
    
    titles = df.select('title')
    titles.show()
    #train, test = df.randomSplit([0.75, 0.25], seed=42)
    

    #hashingTF = HashingTF()
    #tf = hashingTF.transform(titles)

# # While applying HashingTF only needs a single pass to the data, applying IDF needs two passes:
# # First to compute the IDF vector and second to scale the term frequencies by IDF.
# tf.cache()
# idf = IDF().fit(tf)
# tfidf = idf.transform(tf)
    

In [1]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, RegexTokenizer, StopWordsRemover, HashingTF, IDF, CountVectorizer
from pyspark.ml import Pipeline
from pyspark import SparkContext, SparkConf



In [55]:
# Only enter this block if we're in main
if __name__ == "__main__":

    # Create the spark session object
    spark = SparkSession.builder \
    .appName("Spark NLP")\
    .master("local[4]")\
    .config("spark.driver.memory","32G")\
    .config("spark.driver.maxResultSize", "0") \
    .config("spark.kryoserializer.buffer.max", "2000M").getOrCreate()
    
    spark.sparkContext.setLogLevel("WARN")
    sc = spark.sparkContext
    
    
    df = spark.read.csv('Eluvio_DS_Challenge.csv', schema='time_created INT, date_created DATE, up_votes INT, down_votes INT, title STRING, over_18 BOOLEAN, author STRING, category STRING', header=True)
    titles = df.select('title')
    titles.show(truncate=False)
    
    regex = RegexTokenizer(inputCol="title", outputCol="regex", pattern = "\s+")
    stopwords = StopWordsRemover(inputCol="regex", outputCol='filtered')
    pipeline = Pipeline(stages=[regex, stopwords])
    
    
    res = pipeline.fit(titles).transform(titles)#.select('filtered')#.show(truncate=False)
    res = res.na.drop(subset=["filtered"])
    res.show()    
    
    temp = res.select('filtered').rdd.flatMap(lambda x: x)
    temp.collect()
    #rdd = sc.parallelize(temp)
    #title = rdd.map(lambda x: (x, )).toDF(['title'])
    
#     temp = df.select('title').rdd.flatMap(lambda x: x).collect()
#     
    
#     

    
    
    #CountVectorizer(inputCol="title", outputCol="features").fit(title)
    
#     titles = df.select('title')
#     titles.show(truncate=False)

    
#     cv = CountVectorizer(inputCol="filtered", outputCol="features")


    #tf = HashingTF(inputCol="filtered", outputCol="features")
    #model = tf.transform(res).repartition(1000)#.select('features').show(truncate=False)
    
    #print(model.rdd.getNumPartitions())
    #idf = IDF(inputCol='features', outputCol='tfidf')
    #res = idf.fit(model)
#     result = model.transform(res)
#     result.show(truncate=False)
    # Call our main routine
    #main(spark)

+-------------------------------------------------------------------------------------------------+
|title                                                                                            |
+-------------------------------------------------------------------------------------------------+
|Scores killed in Pakistan clashes                                                                |
|Japan resumes refuelling mission                                                                 |
|US presses Egypt on Gaza border                                                                  |
|Jump-start economy: Give health care to all                                                      |
|Council of Europe bashes EU&UN terror blacklist                                                  |
|Hay presto! Farmer unveils the  illegal  mock-Tudor castle he tried to hide behind 40ft hay bales|
|Strikes, Protests and Gridlock at the Poland-Ukraine Border                                      |


Traceback (most recent call last):
  File "/usr/local/Cellar/apache-spark/3.1.1/libexec/python/pyspark/serializers.py", line 437, in dumps
    return cloudpickle.dumps(obj, pickle_protocol)
  File "/usr/local/Cellar/apache-spark/3.1.1/libexec/python/pyspark/cloudpickle/cloudpickle_fast.py", line 101, in dumps
    cp.dump(obj)
  File "/usr/local/Cellar/apache-spark/3.1.1/libexec/python/pyspark/cloudpickle/cloudpickle_fast.py", line 540, in dump
    return Pickler.dump(self, obj)
  File "/Users/JARVIS/opt/anaconda3/lib/python3.7/pickle.py", line 437, in dump
    self.save(obj)
  File "/Users/JARVIS/opt/anaconda3/lib/python3.7/pickle.py", line 504, in save
    f(self, obj) # Call unbound method with explicit self
  File "/Users/JARVIS/opt/anaconda3/lib/python3.7/pickle.py", line 789, in save_tuple
    save(element)
  File "/Users/JARVIS/opt/anaconda3/lib/python3.7/pickle.py", line 504, in save
    f(self, obj) # Call unbound method with explicit self
  File "/usr/local/Cellar/apache-spark

PicklingError: Could not serialize object: Exception: It appears that you are attempting to reference SparkContext from a broadcast variable, action, or transformation. SparkContext can only be used on the driver, not in code that it run on workers. For more information, see SPARK-5063.

In [36]:
from bpe import Encoder

encoder = Encoder(200, pct_bpe=0.88)
encoder.fit(df.title.values)

example = df.title[:10]

print(example.map(encoder.tokenize))
print(next(encoder.transform(example.values)))
print(list(encoder.inverse_transform(encoder.transform(example))))
# print(encoder.tokenize(example))
# print(next(encoder.transform([example])))
# print(next(encoder.inverse_transform(encoder.transform([example]))))


0    [__sow, s, co, re, s, __eow, __sow, ki, ll, ed...
1    [__sow, j, ap, an, __eow, __sow, re, su, me, s...
2    [__sow, us, __eow, __sow, pr, es, se, s, __eow...
3    [__sow, j, u, m, p, __eow, -, __sow, st, ar, t...
4    [__sow, co, un, ci, l, __eow, of, __sow, e, ur...
5    [__sow, ha, y, __eow, __sow, pr, es, to, __eow...
6    [__sow, st, ri, ke, s, __eow, ,, __sow, pr, ot...
7    [the, __sow, u, __eow, ., __sow, n, __eow, ., ...
8    [__sow, ni, co, la, s, __eow, __sow, sa, r, k,...
9    [__sow, us, __eow, __sow, pl, an, s, __eow, fo...
Name: title, dtype: object
[24, 9, 78, 48, 9, 25, 24, 158, 90, 62, 25, 4, 24, 121, 158, 51, 46, 25, 24, 196, 17, 94, 9, 25]
['scores killed in pakistan clashes', 'japan resumes refuelling mission', 'us presses egypt on gaza border', 'jump - start economy : give health care to all', 'council of europe bashes eu __unk un terror blacklist', 'hay presto __unk farmer unveils the illegal mock - tudor castle he tried to hide behind __unk0ft hay bales', 